In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def add_conv_block(input_layer, n_filters_1, n_filters_2, dropout_share):
    conv_1 = tf.layers.conv2d(input_layer, filters=n_filters_1, kernel_size=[3, 3], 
                                    strides=1, padding="same", activation=tf.nn.relu)
    drop_1 = tf.nn.dropout(conv_1, keep_prob=1-dropout_share)
    conv_2 = tf.layers.conv2d(drop_1, filters=n_filters_2, kernel_size=[3, 3], 
                                    strides=1, padding="same", activation=tf.nn.relu)
    drop_2 = tf.nn.dropout(conv_2, keep_prob=1-dropout_share)
    pool_1 = tf.layers.max_pooling2d(drop_2, pool_size=2, strides=2)
    return pool_1

def undo_conv_block(input_layer, n_filters_1, n_filters_2, dropout_share):
    resize_1 = tf.image.resize_images(input_layer, 
                                      2 * np.array(input_layer.get_shape().as_list()[1:3]), 
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    conv_1 = tf.layers.conv2d_transpose(resize_1, filters=n_filters_1, kernel_size=[3, 3], 
                                    strides=1, padding="same", activation=tf.nn.relu)
    drop_1 = tf.nn.dropout(conv_1, keep_prob=1-dropout_share)
    conv_2 = tf.layers.conv2d_transpose(drop_1, filters=n_filters_2, kernel_size=[3, 3], 
                                    strides=1, padding="same", activation=tf.nn.relu)
    drop_2 = tf.nn.dropout(conv_2, keep_prob=1-dropout_share)    
    return drop_2

In [ ]:
tf.reset_default_graph()
dropout_share = tf.placeholder(tf.float32, shape=[])

test_input = tf.constant(np.zeros((10, 160, 320, 3)), dtype=tf.float32)
conv_block_1 = conv_block(test_input, 16, 32, dropout_share)
conv_block_2 = conv_block(conv_block_1, 32, 64, dropout_share)
conv_block_3 = conv_block(conv_block_2, 64, 128, dropout_share)
conv_block_4 = conv_block(conv_block_3, 128, 256, dropout_share)
conv_block_5 = conv_block(conv_block_4, 256, 512, dropout_share)

encoder_flat = tf.contrib.layers.flatten(conv_block_5)

compression = tf.layers.dense(encoder_flat, 100)

decoder_flat = tf.layers.dense(compression, encoder_flat.shape[1], activation=tf.nn.relu)

deconv_block_5 = tf.reshape(decoder_flat, tf.shape(conv_block_5))
deconv_block_4 = undo_conv_block(deconv_block_5, 256, 256, dropout_share)
deconv_block_3 = undo_conv_block(deconv_block_4, 128, 128, dropout_share)
deconv_block_2 = undo_conv_block(deconv_block_3, 64, 64, dropout_share)
deconv_block_1 = undo_conv_block(deconv_block_2, 32, 32, dropout_share)
restored_input = undo_conv_block(deconv_block_1, 16, 3, dropout_share)

print test_input, conv_block_1, conv_block_2, conv_block_3, conv_block_4, conv_block_5
print encoder_flat, compression, decoder_flat
print conv_block_5, conv_block_4, conv_block_3, conv_block_2, conv_block_1, restored_input

In [ ]:
tf.trainable_variables()

In [ ]:
def count_parameters(tf_var):
    return reduce(lambda x, y: x * y, map(int, tf_var.shape))

sum([count_parameters(x) for x in tf.trainable_variables()])